In [ ]:
import os
import sys
import numpy as np
import warnings
from astropy.coordinates import Angle
from astropy.io import fits
from astropy.modeling import models, fitting
from astropy.time import Time
from datetime import date
import math
import glob
import matplotlib.pyplot as plt
import configparser
import csv
from dotenv import load_dotenv
import logging
import pandas as pd
%matplotlib inline
from astropy import constants as const

In [ ]:
from modules.radial_velocity.src.alg import RadialVelocityAlg
from modules.radial_velocity.src.alg_rv_init import RadialVelocityAlgInit
from modules.barycentric_correction.src.alg_barycentric_corr import BarycentricCorrectionAlg
load_dotenv()
TEST_DIR = os.getenv('KPFPIPE_TEST_DATA') 
print('TEST_DIR:', TEST_DIR)
FIT_G = fitting.LevMarLSQFitter()
LIGHT_SPEED = 299792.458   # light speed in km/s
class DotDict(dict):
    pass
MODULE_DIR = '../../modules/radial_velocity/'

In [ ]:
from barycorrpy import get_BC_vel
from astropy.utils import iers

iers_url = iers.IERS_A_URL
iers.Conf.iers_auto_url.set(iers_url)

LIGHT_SPEED_M = const.c.value 

bc_config = '../../modules/barycentric_correction/configs/default.cfg'
config = configparser.ConfigParser()
config.read(bc_config)

jd = 2458834.611148768
# jd = 2458834.607079398

bc_init = BarycentricCorrectionAlg(None, config=config)
print(bc_init.obs_config)
obs_config = bc_init.obs_config

print("")
bc = get_BC_vel(JDUTC=jd,
                ra=obs_config[BarycentricCorrectionAlg.RA],
                dec=obs_config[BarycentricCorrectionAlg.DEC],
                pmra=obs_config[BarycentricCorrectionAlg.PMRA],
                pmdec=obs_config[BarycentricCorrectionAlg.PMDEC],
                px=obs_config[BarycentricCorrectionAlg.PX],
                lat=obs_config[BarycentricCorrectionAlg.LAT],
                longi=obs_config[BarycentricCorrectionAlg.LON],
                alt=obs_config[BarycentricCorrectionAlg.ALT],
                rv=obs_config[BarycentricCorrectionAlg.RV])
print(bc)
print(bc[0][0]/LIGHT_SPEED_M)
# bc = BarycentricCorrectionAlg.get_bc_corr_period(bc_init.obs_config, jd)


In [ ]:
# copy of start_logger from logger.py
def get_level(lvl:str) -> int:
    if lvl == 'debug': return logging.DEBUG
    elif lvl == 'info': return logging.INFO
    elif lvl == 'warning': return logging.WARNING
    elif lvl == 'error': return logging.ERROR
    elif lvl == 'critical': return logging.CRITICAL
    else: return logging.NOTSET

def start_logger(logger_name: str, config: str):
    if config is None: 
        # a config file is not provided, so don't start logger
        print('[{}] missing log configuration...not starting a new logger'.format(
            logger_name))
        return None
    config_obj = configparser.ConfigParser()
    res = config_obj.read(config)
    if res == []:
        return None

    log_cfg = config_obj['LOGGER']

    log_start = log_cfg.get('start_log', False)
    log_path = log_cfg.get('log_path', 'log')
    log_lvl = log_cfg.get('log_level', logging.WARNING)
    log_verbose = log_cfg.getboolean('log_verbose', True)
    # logger.setLevel(get_level(log_lvl))
        
    # if log_start:
    #     # setup a log format
    #     formatter = logging.Formatter('[%(name)s][%(levelname)s]:%(message)s')
    #     # setup a log file
    #     f_handle = logging.FileHandler(log_path, mode='w') # logging to file
    #     f_handle.setLevel(get_level(log_lvl))
    #     f_handle.setFormatter(formatter)
    #     logger.addHandler(f_handle)

    #     if log_verbose: 
    #         # also print to terminal 
    #         s_handle = logging.StreamHandler()
    #         s_handle.setLevel(get_level(log_lvl))
    #         s_handle.setFormatter(formatter)
    #         logger.addHandler(s_handle)
    # return logger


    logger = logging.getLogger(logger_name)
    logger.setLevel(get_level(log_lvl))
    logger.propagate = False

    formatter = logging.Formatter('[%(name)s][%(levelname)s]:%(message)s')
    s_handle = logging.StreamHandler()
    s_handle.setLevel(get_level(log_lvl))
    s_handle.setFormatter(formatter)
    logger.addHandler(s_handle)
    return logger


## barycentric correction  from alg_rv_init

In [ ]:
# 1. from RV init
config_file = MODULE_DIR + 'configs/default.cfg'
config = configparser.ConfigParser()
config.read(config_file)

logger = start_logger("OrderTraceAlg", config_file)

# bc_time = Time(date.today().strftime("%Y-%m-%d")).jd - period

bc_time = Time("2019-04-18").jd
corr_path = '../../modules/barycentric_correction/results/'
bc_output = corr_path + "bc_from_rv_init.csv"
rv_init = RadialVelocityAlgInit(config, logger, bc_time=bc_time, bc_corr_path=corr_path, bc_corr_output=bc_output)
init_result = rv_init.start(print_debug='')


In [ ]:
# 2. from barycentric correction module
bc_config = '../../modules/barycentric_correction/configs/default.cfg'
config = configparser.ConfigParser()
config.read(bc_config)

bc_init = BarycentricCorrectionAlg(None, config=config)
corr_path = '../../modules/barycentric_correction/results/'
# bc_time  =Time("2019-04-18").jd
bc_time = 2458834.611148768
period = None
#bc_output = corr_path+"bc_from_alg_no_rv.csv"
bc_output = None
print(bc_init.obs_config)
#zb_range = bc_init.get_zb_long(bc_time, period, data_path=corr_path, save_to_path=bc_output)
zb_list = bc_init.get_zb_list(bc_time, period, data_path=corr_path, save_to_path=bc_output )
print(zb_list)